In [26]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [27]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\output data'
name_DD_M = r'dataASCD_M.xlsx'
name_DD_P = r'dataASCD_P.xlsx'

asset_data = 'AssetData.csv'
asset_system = 'AssetStructureA&S.csv'
system_component = 'AssetStructureS&C.csv'
component_dutie = 'DutieStructureC&D.csv'
dutie_resource = 'DutieStructureD&R.csv'

ruteA_D = os.path.join(folder,asset_data)
ruteA_S = os.path.join(folder,asset_system)
ruteS_C = os.path.join(folder,system_component)
ruteC_D = os.path.join(folder,component_dutie)
ruteD_D = os.path.join(folder,dutie_data)
ruteD_R = os.path.join(folder,dutie_resource)

ruteDD_M = os.path.join(folder,name_DD_M)
ruteDD_P = os.path.join(folder,name_DD_P)



In [28]:
dataASCD_M = pd.read_excel(ruteDD_M, header=0)
dataASCD_P = pd.read_excel(ruteDD_P, header=0)

In [ ]:
dataA_D_S = pd.merge(dataA_D_JJ,dataA_S,how = 'left',on = 'Asset tag')
dataA_S_C = pd.merge(dataA_D_S, dataS_C, how='left', on = 'key_system')
dataA_S_C_D = pd.merge(dataA_S_C, dataC_D, how='left', on='key_component')

dataASCD_M = pd.merge(dataA_S_C_D, dataD_D_M_1T, how = 'inner', on = 'key_dutie')
dataASCD_P = pd.merge(dataA_S_C_D, dataD_D_P_1T, how = 'inner', on = 'key_dutie')

# Saving the data
dataASCD_M.to_excel('dataASCD_M.xlsx',encoding='utf-8-sig', index=False)
dataASCD_P.to_excel('dataASCD_P.xlsx',encoding='utf-8-sig', index=False)

In [30]:
%%time
# Adding the pivot column to project
dataAllcompileM.loc[:,'pivot'] = dataAllcompileM.loc[:,'Asset tag'] + '-' + dataAllcompileM.loc[:,'Strategy'] 

Asset tag                          0
Name                              23
Cod sap                          270
WBS_1_EN                           0
WBS_2_EN                           0
WBS_3_EN                           0
WBS_4_EN                           0
Responsable                        0
Hour day                           0
Criticidad                         0
Criticidad Name                    0
key_system                         0
System                          3276
key_component                      0
Component                          0
key_dutie                          0
Primary task                       0
Primary task detail activity       0
Maintenance type                   0
Specialist                         0
Constraint                         0
Time                               0
Amount Craft                       0
Frequency                          0
Unidad Periodo                     0
Last date task done                0
Man hour                           0
S